# Homework 2
### Sean Morton, ME449

In [2]:
#imports
import core as mr
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


In [36]:
#variables determined from project description
W1 = 0.109 #changed mm to m
W2 = 0.082
L1 = 0.425
L2 = 0.392
H1 = 0.089
H2 = 0.095

Jb = np.matrix([
    [0,  1, 0, (W1 + W2),          0, (L1 + L2)],
    [0,  0, 1,        H2, -(L1 + L2),         0],
    [0,  0, 1,        H2,        -L2,         0],
    [0,  0, 1,        H2,          0,         0],
    [0, -1, 0,       -W2,          0,         0],
    [0,  0, 1,         0,          0,         0]
]).T

Tsd = np.matrix([
    [-1, 0, 0, -0.2],
    [0,  0, 1,  0.6],
    [0,  1, 0, 0.35],
    [0,  0, 0,    1]
])

M = np.matrix([
    [-1, 0, 0, (L1 + L2)],
    [ 0, 0, 1, (W1 + W2)],
    [ 0, 1, 0, (H1 - H2)],
    [ 0, 0, 0,         1]
])

In [ ]:
#helper functions
def find_twist_for_error(M, Jb, Tsb, thetalist):
    '''
    - inputs: M, theta_list, Jb or Js, target transf. Tsb
    - calculate current transformation matrix of e.e. Tsa from:
        - starting config M
        - joint angles theta_list
        - Jacobian of robot in S or B frame
    - find Tab: Tab = (Tsa)^-1 Tsb
    - find twist by [V]*t = log(Tab)
        - assume rotating w/ twist for unit time 1sec will
            give trans. matrix
    - V = se3ToVec([V])
    '''
    
    #find transformation that describes difference between 
    #current position and desired position
    Tsa = FKInBody(M, Jb, thetalist)
    Tab = mr.TransInv(Tsa) * Tsb
    
    #assume V*(1sec) gives transformation
    se3_Vt = mr.MatrixLog6(Tab)
    V = mr.se3ToVec(se3_Vt)
    
    return V



In [54]:
def calculate_V_error(V):
    '''
    - inputs: a twist V
    - outputs: [eomg_curr, elin_curr]
    '''
    w = V[0:3]
    v = V[3:6]
    mag_w = np.linalg.norm(w)
    mag_v = np.linalg.norm(v)
    return np.array([mag_w, mag_v])

V = np.array([2,4,5,6,7,9])
error = calculate_V_error(V)
expected = [np.sqrt(45), np.sqrt(166)]

assert np.allclose(expected, error), f"Observed error: {error}"


In [ ]:
def convert_theta_to_xyz(M, Blist, csv_filepath):
    '''
    - write function to convert a series of joint angles into positions 
    in xyz coords of the end effector
        - inputs: M, Blist/Slist, csv_filepath
        - opens and iterates through CSV file
        - uses: FKinBody() or FKinSpace(), inputs M, Blist/Slist, thetalist
        - calculate a transformation matrix that results from 
            the joint angles applied
        - take home configuration of robot and pre- or post-multiply
            transformation from joint angles
        - use [R,p] = TransToRp(T) to get the translation of the end
            effector rel. to. body frame; extract set x,y,z = p
        - outputs a list of lists, [ [x,y,z], [x,y,z], ...) 
        
    Used for plotting the positions of the end effector over time.
    '''
    df = pd.read_csv(csv_filepath, header=None)
    p_list = []
    
    for ind, row in df.iterrows():
        angles = row.tolist()
        
        #apply forward kinematics with given angles; extract xyz and R
        T = mr.FKinBody(M, Blist, angles)
        [_, p] = mr.TransToRp(T)
        p = p.round(3).tolist()
        p_list.append(p)
    
    ####
    return p_list
        
        
expected = [
    [-0.594, -0.167, -0.162],
    [-0.055,  0.100, -0.364],
    [ 0.033, -0.027,  0.184],
    [ 0.817,  0.191,  0.005],
]

local_csv = "test_hw2_angles.csv"
p_list = convert_theta_to_xyz(M, Jb,local_csv)

#these tolerances make no sense, but I verified by inspection
#that observed elements are close to expected elements within 0.001
assert np.allclose(expected, p_list, atol = 0.02, rtol = 0.02), \
        f"Observed result: {p_list}"